In [18]:
# for suppress warnings
# python packages
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.simplefilter("ignore")

In [59]:
import os
os.environ["JAVA_HOME"] = "/Program Files/Java/jdk1.8.0_281"
os.environ["SPARK_HOME"] = "/Users/rishnamc/Documents/Maveric/Spark/spark-3.1.2-bin-hadoop2.7"

import findspark
findspark.init()

import glob as g
import datetime
from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark import SparkContext
import pyspark.sql.functions as f
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import *

sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))
spark = SparkSession.builder.appName("YouTube Data Analysis").getOrCreate()
print(spark.sparkContext.appName)

pyspark-shell


In [61]:
mySchema = StructType([ StructField("video_id", StringType(), True)\
                       ,StructField("trending_date", DateType(), True)\
                       ,StructField("title", StringType(), True)\
                       ,StructField("channel_title", StringType(), True)\
                       ,StructField("category_id", IntegerType(), True)\
                       ,StructField("publish_time", TimestampType(), True)\
                       ,StructField("tags", StringType(), True)\
                       ,StructField("views", IntegerType(), True)\
                       ,StructField("likes", IntegerType(), True)\
                       ,StructField("dislikes", IntegerType(), True)\
                       ,StructField("comment_count", IntegerType(), True)\
                       ,StructField("thumbnail_link", StringType(), True)\
                       ,StructField("comments_disabled", StringType(), True)\
                       ,StructField("ratings_disabled", StringType(), True)\
                       ,StructField("video_error_or_removed", StringType(), True)\
                       ,StructField("description", StringType(), True)])
mySchema

StructType(List(StructField(video_id,StringType,true),StructField(trending_date,DateType,true),StructField(title,StringType,true),StructField(channel_title,StringType,true),StructField(category_id,IntegerType,true),StructField(publish_time,TimestampType,true),StructField(tags,StringType,true),StructField(views,IntegerType,true),StructField(likes,IntegerType,true),StructField(dislikes,IntegerType,true),StructField(comment_count,IntegerType,true),StructField(thumbnail_link,StringType,true),StructField(comments_disabled,StringType,true),StructField(ratings_disabled,StringType,true),StructField(video_error_or_removed,StringType,true),StructField(description,StringType,true)))

In [62]:
paths = g.glob("*.csv", recursive=True)
all_csv = spark.read.csv(paths, header=True)

In [21]:
all_csv.show(5)
all_csv.printSchema()

+-----------+-------------+--------------------+-------------+-----------+--------------------+--------------------+------+-----+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|   video_id|trending_date|               title|channel_title|category_id|        publish_time|                tags| views|likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|video_error_or_removed|         description|
+-----------+-------------+--------------------+-------------+-----------+--------------------+--------------------+------+-----+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|gDuslQ9avLc|     17.14.11|Захар и Полина уч...|    Т—Ж БОГАЧ|         22|2017-11-13T09:09:...|"захар и полина|"...| 62408|  334|     190|           50|https://i.ytimg.c...|            FALSE|           FALSE|                 FALSE|Знакомьтес

In [5]:
print("Shape of the records in csv file",all_csv.count(),len(all_csv.columns))

Shape of the records in csv file 416869 16


In [63]:
#Remove irrelevant columns.
all_csv=all_csv.drop('thumbnail_link','comments_disabled','ratings_disabled','video_error_or_removed','description')

In [23]:
all_csv.describe()

DataFrame[summary: string, video_id: string, trending_date: string, title: string, channel_title: string, category_id: string, publish_time: string, tags: string, views: string, likes: string, dislikes: string, comment_count: string]

In [24]:
all_csv.dtypes

[('video_id', 'string'),
 ('trending_date', 'string'),
 ('title', 'string'),
 ('channel_title', 'string'),
 ('category_id', 'string'),
 ('publish_time', 'string'),
 ('tags', 'string'),
 ('views', 'string'),
 ('likes', 'string'),
 ('dislikes', 'string'),
 ('comment_count', 'string')]

In [25]:
all_csv.summary().show(5)

+-------+--------------------+------------------+------------------+--------------------+------------------+------------------+------------------+------------------+------------------+--------------------+-----------------+
|summary|            video_id|     trending_date|             title|       channel_title|       category_id|      publish_time|              tags|             views|             likes|            dislikes|    comment_count|
+-------+--------------------+------------------+------------------+--------------------+------------------+------------------+------------------+------------------+------------------+--------------------+-----------------+
|  count|              416869|            381271|            379782|              379120|            378301|            377845|            376784|            376598|            376519|              376462|           376448|
|   mean|                 1.0|2073.7291666666665| 472.6363636363636|            Infinity|20.232380354520

In [10]:
all_csv.show(5)

+-----------+-------------+--------------------+-------------+-----------+--------------------+--------------------+------+-----+--------+-------------+
|   video_id|trending_date|               title|channel_title|category_id|        publish_time|                tags| views|likes|dislikes|comment_count|
+-----------+-------------+--------------------+-------------+-----------+--------------------+--------------------+------+-----+--------+-------------+
|gDuslQ9avLc|     17.14.11|Захар и Полина уч...|    Т—Ж БОГАЧ|         22|2017-11-13T09:09:...|"захар и полина|"...| 62408|  334|     190|           50|
|AOCJIFEA_jE|     17.14.11|Биржа Мемов #29. ...| Druzhko Show|         22|2017-11-13T17:32:...|"биржа мемов|""ле...|330043|43841|    2244|         2977|
|VAWNQDgwwOM|     17.14.11|ХАЙП КЭМП - СВОЙ ...|         Юлик|         24|2017-11-13T16:11:...|"юмор|""комедия""...|424596|49854|     714|         2944|
|gknkFwKQfHg|     17.14.11|Сочная кесадилья ...|     Hochland|         22|2017-11-

In [11]:
all_csv.toPandas()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count
0,gDuslQ9avLc,17.14.11,Захар и Полина учатся экономить,Т—Ж БОГАЧ,22,2017-11-13T09:09:31.000Z,"""захар и полина|""""учимся экономить""""|""""копить ...",62408,334,190,50
1,AOCJIFEA_jE,17.14.11,Биржа Мемов #29. Большой выпуск,Druzhko Show,22,2017-11-13T17:32:11.000Z,"""биржа мемов|""""лев шагинян""""|""""мемы""""|""""пикчи""...",330043,43841,2244,2977
2,VAWNQDgwwOM,17.14.11,ХАЙП КЭМП - СВОЙ СРЕДИ ЧУЖИХ,Юлик,24,2017-11-13T16:11:31.000Z,"""юмор|""""комедия""""|""""влог""""|""""блог""""|""""смешно""""...",424596,49854,714,2944
3,gknkFwKQfHg,17.14.11,Сочная кесадилья с курицей,Hochland,22,2017-11-13T06:51:10.000Z,"""хохланд|""""сыр""""|""""рецепты""""|""""как приготовить...",112851,3566,122,80
4,3sYvZcwzZr8,17.14.11,КЛИПЫ РОДИТЕЛЕЙ НА ШКОЛЬНЫЙ ВЫПУСКНОЙ,Совергон,24,2017-11-13T16:52:36.000Z,"""Совергон|""""Sovergon""""|""""клипы""""|""""родители""""|...",243469,36216,631,1692
...,...,...,...,...,...,...,...,...,...,...,...
416864,OMmR9THjVKM,18.14.06,У меня СДВГ! (Story booth на русском) Русская ...,Pastime Time,22,2018-06-13T13:47:01.000Z,"""story booth на русском|""""story booth перевод""...",129488,5893,164,990
416865,tX7p7NtNVDE,18.14.06,Мои Детские Травмы 2 (анимация),CaGArt,23,2018-06-12T09:38:38.000Z,"""кагарт|""""каги""""|""""цагарт""""|""""гагарт""""|""""какар...",99163,4659,337,692
416866,KAyj5Xm1C64,18.14.06,[ENG SUB] BTS PROM PARTY 2018 Intro + 2nd Gran...,DaisyxBTS 07,24,2018-06-13T12:51:23.000Z,[none],449611,24808,93,974
416867,4PiSLIrsSiY,18.14.06,ОБЗОР ВАННОЙ КОМНАТЫ🛁/ТУАЛЕТНОЙ КОМНАТЫ🚽 + ДЕК...,Ксюша Лебедева,22,2018-06-13T00:23:33.000Z,"""рум тур ванной комнаты|""""room tour bathroom""""...",14225,793,39,209


In [12]:
#all_csv=all_csv.withColumn("country",lit("country")).show(5)

In [13]:
all_csv.printSchema()

root
 |-- video_id: string (nullable = true)
 |-- trending_date: string (nullable = true)
 |-- title: string (nullable = true)
 |-- channel_title: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- publish_time: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- views: string (nullable = true)
 |-- likes: string (nullable = true)
 |-- dislikes: string (nullable = true)
 |-- comment_count: string (nullable = true)



In [14]:
all_dfs = [] # list to store each data frame separately
for csv in allcsv:
    df = pd.read_csv(csv)
    df['country'] = csv[0:2] # adding column 'country' so that each dataset could be identified uniquely
    all_dfs.append(df)
all_dfs[0].head()

NameError: name 'allcsv' is not defined

In [64]:
all_csv=all_csv.withColumn("publish_date",to_date(all_csv.publish_time))
all_csv.select('publish_date')
all_csv.show(5)

+-----------+-------------+--------------------+-------------+-----------+--------------------+--------------------+------+-----+--------+-------------+------------+
|   video_id|trending_date|               title|channel_title|category_id|        publish_time|                tags| views|likes|dislikes|comment_count|publish_date|
+-----------+-------------+--------------------+-------------+-----------+--------------------+--------------------+------+-----+--------+-------------+------------+
|gDuslQ9avLc|     17.14.11|Захар и Полина уч...|    Т—Ж БОГАЧ|         22|2017-11-13T09:09:...|"захар и полина|"...| 62408|  334|     190|           50|  2017-11-13|
|AOCJIFEA_jE|     17.14.11|Биржа Мемов #29. ...| Druzhko Show|         22|2017-11-13T17:32:...|"биржа мемов|""ле...|330043|43841|    2244|         2977|  2017-11-13|
|VAWNQDgwwOM|     17.14.11|ХАЙП КЭМП - СВОЙ ...|         Юлик|         24|2017-11-13T16:11:...|"юмор|""комедия""...|424596|49854|     714|         2944|  2017-11-13|
|gkn

In [29]:
all_csv.dtypes

[('video_id', 'string'),
 ('trending_date', 'string'),
 ('title', 'string'),
 ('channel_title', 'string'),
 ('category_id', 'string'),
 ('publish_time', 'string'),
 ('tags', 'string'),
 ('views', 'string'),
 ('likes', 'string'),
 ('dislikes', 'string'),
 ('comment_count', 'string'),
 ('publish_date', 'date')]

In [30]:
df=all_csv.select('publish_time')
df.show(5)

+--------------------+
|        publish_time|
+--------------------+
|2017-11-13T09:09:...|
|2017-11-13T17:32:...|
|2017-11-13T16:11:...|
|2017-11-13T06:51:...|
|2017-11-13T16:52:...|
+--------------------+
only showing top 5 rows



In [ ]:
#all_csv.withColumn('publishtime',min("$publishtime"))

In [65]:
all_csv=all_csv.withColumn('publishtime',to_timestamp(all_csv.publish_time))
all_csv.select('publishtime')
all_csv.show(5)

+-----------+-------------+--------------------+-------------+-----------+--------------------+--------------------+------+-----+--------+-------------+------------+-------------------+
|   video_id|trending_date|               title|channel_title|category_id|        publish_time|                tags| views|likes|dislikes|comment_count|publish_date|        publishtime|
+-----------+-------------+--------------------+-------------+-----------+--------------------+--------------------+------+-----+--------+-------------+------------+-------------------+
|gDuslQ9avLc|     17.14.11|Захар и Полина уч...|    Т—Ж БОГАЧ|         22|2017-11-13T09:09:...|"захар и полина|"...| 62408|  334|     190|           50|  2017-11-13|2017-11-13 14:39:31|
|AOCJIFEA_jE|     17.14.11|Биржа Мемов #29. ...| Druzhko Show|         22|2017-11-13T17:32:...|"биржа мемов|""ле...|330043|43841|    2244|         2977|  2017-11-13|2017-11-13 23:02:11|
|VAWNQDgwwOM|     17.14.11|ХАЙП КЭМП - СВОЙ ...|         Юлик|        

In [67]:
all_csv.printSchema()

root
 |-- video_id: string (nullable = true)
 |-- trending_date: string (nullable = true)
 |-- title: string (nullable = true)
 |-- channel_title: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- publish_time: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- views: string (nullable = true)
 |-- likes: string (nullable = true)
 |-- dislikes: string (nullable = true)
 |-- comment_count: string (nullable = true)
 |-- publish_date: date (nullable = true)
 |-- publishtime: timestamp (nullable = true)



In [66]:
df1=all_csv.select('trending_date')
df1.show(5)

+-------------+
|trending_date|
+-------------+
|     17.14.11|
|     17.14.11|
|     17.14.11|
|     17.14.11|
|     17.14.11|
+-------------+
only showing top 5 rows



In [41]:
all_csv.createOrReplaceTempView("youtube")

In [48]:
spark.sql("select DATE_FORMAT(trending_date,'ddMMyyyy') from youtube")

DataFrame[date_format(CAST(trending_date AS TIMESTAMP), ddMMyyyy): string]

In [40]:
all_csv.createOrReplaceTempView("youtube")
df1=spark.createDataFrame(['trending_date'])
#df= spark.sql(select "trending_date"(convert(varchar, getdate(), 23)) as "trending_date" FROM 'youtube')

TypeError: Can not infer schema for type: <class 'str'>

In [ ]:
#all_csv.withColumn("trendingdate", f.format_string("20%d","trendingdate")).show(5)
#a=20
#b=col("trending_date")
#all_csv.concat_ws('20', "trending_date"): Column	
all_csv.select(col("trending_date"),concat("20%d",lit(""),col("trending_date")))
#all_csv.select("20", concat(col("trending_date").to_date(all_csv.trending_date))).show()

In [ ]:
all_csv=all_csv.select(date_format(col('trending_date'),"yyyy-MM-dd").alias('trending_date').cast("date"))
all_csv.show(10,False)

In [ ]:
from pyspark.sql import functions as sf
all_csv.withColumn('trending_date',sf.concat(sf.lit('20'), sf.col('trending_date'))).show()


In [ ]:
cor_rel=all_csv.corr("views","likes","pearson")
print("positive relationship : %a" %cor_rel)